# Module-SIS and Module-LWE: The Goldilocks Zone

Ring versions are super efficient but maybe too structured. Regular versions are secure but too slow. Module versions? Just right.

**Credits to:**  
Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

---

**What we'll cover:**
- Modules: vectors of polynomials
- Module-SIS (MSIS): somewhere between SIS and Ring-SIS
- Module-LWE (MLWE): somewhere between LWE and Ring-LWE
- Kyber encryption: real-world MLWE crypto
- Why this balance is perfect for practice

## The Module Idea

Remember the progression?
- **SIS/LWE:** Random matrices $A$ (slow, big)
- **Ring-SIS/Ring-LWE:** Single polynomial $a$ expands to structured matrix (fast, small)
- **Module versions:** Vectors of polynomials (middle ground)

Instead of one polynomial, we work with vectors in $R_q^k$:

$$
\mathbf{a} = \begin{bmatrix} a_1(x) \\ a_2(x) \\ \vdots \\ a_k(x) \end{bmatrix}
$$

where each $a_i(x) \in R_q = \mathbb{Z}_q[x]/(x^n + 1)$.

Operations:
- **Addition/subtraction:** Component-wise
- **Inner product:** $\mathbf{a}^T \mathbf{b} = a_1b_1 + a_2b_2 + \cdots + a_kb_k$ (result is a polynomial)
- **Size:** $\|\mathbf{a}\|_\infty = \max_i \|a_i\|_\infty$

In [1]:
import numpy as np

# Working with module elements (vectors of polynomials)
q = 17
n = 4
k = 3

# Three polynomials form a vector in R_q^3
a1 = np.array([3, 5, 2, 1])    # 3 + 5x + 2x^2 + x^3
a2 = np.array([7, 0, 4, 3])    # 7 + 4x^2 + 3x^3
a3 = np.array([2, 6, 1, 8])    # 2 + 6x + x^2 + 8x^3

print("Module Element in R_q^3")
print("=" * 60)
print(f"Parameters: q={q}, n={n}, k={k}")
print(f"Ring: R_q = Z_{q}[x]/(x^{n} + 1)")

print(f"\nVector a = [a₁(x), a₂(x), a₃(x)]ᵀ where:")
print(f"  a₁(x) = {a1[0]} + {a1[1]}x + {a1[2]}x² + {a1[3]}x³")
print(f"  a₂(x) = {a2[0]} + {a2[2]}x² + {a2[3]}x³")
print(f"  a₃(x) = {a3[0]} + {a3[1]}x + {a3[2]}x² + {a3[3]}x³")

# Size of the vector
def poly_norm_inf(poly, q):
    """Infinity norm of polynomial (max coefficient in centered rep)"""
    centered = np.where(poly > q//2, poly - q, poly)
    return np.max(np.abs(centered))

norms = [poly_norm_inf(a1, q), poly_norm_inf(a2, q), poly_norm_inf(a3, q)]
norm_a = max(norms)

print(f"\nNorms: ||a₁||∞={norms[0]}, ||a₂||∞={norms[1]}, ||a₃||∞={norms[2]}")
print(f"||a||∞ = max(||aᵢ||∞) = {norm_a}")

print("\nThis is the basic building block for Module-SIS/LWE")

Module Element in R_q^3
Parameters: q=17, n=4, k=3
Ring: R_q = Z_17[x]/(x^4 + 1)

Vector a = [a₁(x), a₂(x), a₃(x)]ᵀ where:
  a₁(x) = 3 + 5x + 2x² + 1x³
  a₂(x) = 7 + 4x² + 3x³
  a₃(x) = 2 + 6x + 1x² + 8x³

Norms: ||a₁||∞=5, ||a₂||∞=7, ||a₃||∞=8
||a||∞ = max(||aᵢ||∞) = 8

This is the basic building block for Module-SIS/LWE


## Module-SIS: The Problem

Module-SIS generalizes Ring-SIS by using vectors of polynomials instead of single polynomials.

**$MSIS(n, k, \ell, q, B)$:** Given $\mathbf{a}_1, \ldots, \mathbf{a}_\ell \in R_q^k$ (where $\ell > k$), find polynomials $z_1, \ldots, z_\ell \in R_q$ such that:

$$
\mathbf{a}_1 z_1 + \mathbf{a}_2 z_2 + \cdots + \mathbf{a}_\ell z_\ell = \mathbf{0}
$$

with $\|z_i\|_\infty \leq B$ and not all $z_i$ are zero.

In matrix form, we're solving:

$$
\begin{bmatrix}
a_{11} & a_{21} & \cdots & a_{\ell 1} \\
a_{12} & a_{22} & \cdots & a_{\ell 2} \\
\vdots & \vdots & \ddots & \vdots \\
a_{1k} & a_{2k} & \cdots & a_{\ell k}
\end{bmatrix}
\begin{bmatrix}
z_1 \\ z_2 \\ \vdots \\ z_\ell
\end{bmatrix}
= \begin{bmatrix}
0 \\ 0 \\ \vdots \\ 0
\end{bmatrix}
$$

Each block expands to an anti-circulant matrix, so the full matrix is $kn \times \ell n$.

In [2]:
# Module-SIS example structure
q = 67
n = 4
k = 2  # Each vector has 2 polynomials
ell = 3  # We have 3 vectors
B = 10

print("Module-SIS Instance Structure")
print("=" * 60)
print(f"Parameters: q={q}, n={n}, k={k}, ℓ={ell}, B={B}")

# Each a_i is a vector of k polynomials
a1_1 = np.array([32, 0, 66, 33])  # First polynomial of a_1
a1_2 = np.array([30, 64, 31, 65])  # Second polynomial of a_1

a2_1 = np.array([42, 44, 20, 65])
a2_2 = np.array([63, 41, 19, 64])

a3_1 = np.array([2, 60, 33, 42])
a3_2 = np.array([26, 9, 57, 7])

print(f"\nGiven {ell} vectors in R_q^{k}:")
print(f"a₁ = [a₁₁(x), a₁₂(x)]ᵀ")
print(f"a₂ = [a₂₁(x), a₂₂(x)]ᵀ")
print(f"a₃ = [a₃₁(x), a₃₂(x)]ᵀ")

print(f"\nGoal: Find z₁, z₂, z₃ with ||zᵢ||∞ ≤ {B} such that:")
print("  a₁₁·z₁ + a₂₁·z₂ + a₃₁·z₃ = 0")
print("  a₁₂·z₁ + a₂₂·z₂ + a₃₂·z₃ = 0")

# Anti-circulant helper
def anti_circulant(a):
    n = len(a)
    C = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i >= j:
                C[i, j] = a[i - j]
            else:
                C[i, j] = -a[n + i - j]
    return C

# Build the full matrix A
A_top = np.hstack([anti_circulant(a1_1), anti_circulant(a2_1), anti_circulant(a3_1)])
A_bot = np.hstack([anti_circulant(a1_2), anti_circulant(a2_2), anti_circulant(a3_2)])
A = np.vstack([A_top, A_bot]) % q

print(f"\nFull matrix A is {k*n}×{ell*n} = {k*n}×{ell*n}")
print(f"A has {k} row blocks, each is anti-circulant matrices")
print(f"Total: {A.shape[0]} rows × {A.shape[1]} columns")

Module-SIS Instance Structure
Parameters: q=67, n=4, k=2, ℓ=3, B=10

Given 3 vectors in R_q^2:
a₁ = [a₁₁(x), a₁₂(x)]ᵀ
a₂ = [a₂₁(x), a₂₂(x)]ᵀ
a₃ = [a₃₁(x), a₃₂(x)]ᵀ

Goal: Find z₁, z₂, z₃ with ||zᵢ||∞ ≤ 10 such that:
  a₁₁·z₁ + a₂₁·z₂ + a₃₁·z₃ = 0
  a₁₂·z₁ + a₂₂·z₂ + a₃₂·z₃ = 0

Full matrix A is 8×12 = 8×12
A has 2 row blocks, each is anti-circulant matrices
Total: 8 rows × 12 columns


## Solving Module-SIS

Just like regular SIS, we can solve MSIS by Gaussian elimination to find the solution space, then search for small solutions.

The approach:
1. Build the full matrix $A$ (size $kn \times \ell n$)
2. Gaussian eliminate to get reduced form
3. Express free variables in terms of constrained ones
4. Search for solutions in $[-B, B]^{\ell n}$

One solution from the example: $z_1(x) = 6 - 8x + 8x^2$, $z_2(x) = 2 + 10x - 6x^2 + 3x^3$, $z_3(x) = -9 + 6x + 3x^2 + 2x^3$

In [3]:
# Verify a Module-SIS solution
# Solution from the example
z1 = np.array([6, -8, 8, 0])
z2 = np.array([2, 10, -6, 3])
z3 = np.array([-9, 6, 3, 2])

z_full = np.concatenate([z1, z2, z3])

print("Module-SIS Solution Verification")
print("=" * 60)

print("Solution polynomials:")
print(f"z₁(x) = 6 - 8x + 8x²")
print(f"z₂(x) = 2 + 10x - 6x² + 3x³")
print(f"z₃(x) = -9 + 6x + 3x² + 2x³")

# Check norms
norms = [poly_norm_inf(z1, q), poly_norm_inf(z2, q), poly_norm_inf(z3, q)]
print(f"\nNorms: ||z₁||∞={norms[0]}, ||z₂||∞={norms[1]}, ||z₃||∞={norms[2]}")
print(f"All ≤ B={B}? {all(norm <= B for norm in norms)}")

# Verify Az = 0 (mod q)
result = (A @ z_full) % q
print(f"\nAz mod {q} = {result[:8]}... (showing first 8)")
all_zero = np.all(result == 0)
print(f"All zeros? {all_zero}")

if all_zero:
    print("\nSolution verified!")
    
# Polynomial multiplication verification
def poly_mult_mod(a, b, q, n):
    """Multiply polynomials mod (x^n + 1) and mod q"""
    A_mat = anti_circulant(a)
    return (A_mat @ b) % q

# Check first equation: a11*z1 + a21*z2 + a31*z3 = 0
term1 = poly_mult_mod(a1_1, z1, q, n)
term2 = poly_mult_mod(a2_1, z2, q, n)
term3 = poly_mult_mod(a3_1, z3, q, n)
eq1 = (term1 + term2 + term3) % q

print(f"\nPolynomial verification:")
print(f"a₁₁·z₁ + a₂₁·z₂ + a₃₁·z₃ = {eq1}")
print(f"Equals zero? {np.all(eq1 == 0)}")

Module-SIS Solution Verification
Solution polynomials:
z₁(x) = 6 - 8x + 8x²
z₂(x) = 2 + 10x - 6x² + 3x³
z₃(x) = -9 + 6x + 3x² + 2x³

Norms: ||z₁||∞=8, ||z₂||∞=10, ||z₃||∞=9
All ≤ B=10? True

Az mod 67 = [0 0 0 0 0 0 0 0]... (showing first 8)
All zeros? True

Solution verified!

Polynomial verification:
a₁₁·z₁ + a₂₁·z₂ + a₃₁·z₃ = [0 0 0 0]
Equals zero? True


## MSIS: The Spectrum

Module-SIS sits between two extremes:

**Setting $k = 1$:** Each vector has just one polynomial, so we get Ring-SIS
- Most structured, fastest
- $A$ is $n \times \ell n$

**Setting $n = 1$:** Polynomials become scalars, so we get regular SIS  
- Least structured, slowest
- $A$ is $k \times \ell$

**Module-SIS with general $k, n$:** The sweet spot
- Matrix size: $kn \times \ell n$
- Can tune $k$ for different security levels while keeping $q, n$ fixed
- Used in Dilithium: $q = 8380417$, $n = 256$, $(k, \ell) \in \{(4,4), (6,5), (8,7)\}$

In [4]:
# Comparing SIS, Ring-SIS, and Module-SIS
n = 256
q = 8380417

print("The SIS/Ring-SIS/Module-SIS Spectrum")
print("=" * 70)
print(f"Fixed: n={n}, q={q}\n")

# Regular SIS
k_sis = 4
ell_sis = 4
sis_size = k_sis * ell_sis
sis_kb = (sis_size * 4) / 1024  # 4 bytes per element

print("1. Regular SIS (k=4, ℓ=4, n=1):")
print(f"   Matrix: {k_sis}×{ell_sis}")
print(f"   Storage: {sis_kb:.1f} KB")
print(f"   Structure: None (random)")

# Ring-SIS
ringsis_polys = 4
ringsis_size = ringsis_polys * n
ringsis_kb = (ringsis_size * 4) / 1024

print(f"\n2. Ring-SIS (ℓ={ringsis_polys}, k=1):")
print(f"   Matrix: {n}×{ringsis_polys * n}")
print(f"   Storage: {ringsis_kb:.1f} KB ({ringsis_polys} polynomials)")
print(f"   Structure: Maximum (anti-circulant)")

# Module-SIS (Dilithium parameters)
for k, ell in [(4, 4), (6, 5), (8, 7)]:
    msis_size = ell * n  # Store ℓ vectors, each with k polynomials
    msis_kb = (k * n * ell * 4) / 1024  # But each polynomial needs storage
    actual_storage_kb = (ell * k * n * 4) / 1024
    
    print(f"\n3. Module-SIS (k={k}, ℓ={ell}):")
    print(f"   Matrix: {k*n}×{ell*n}")
    print(f"   Storage: ~{actual_storage_kb:.1f} KB ({ell} vectors of {k} polynomials)")
    print(f"   Structure: Balanced")

print("\nModule-SIS gives flexible security levels by varying k!")
print("Can optimize polynomial arithmetic for fixed n, q")

The SIS/Ring-SIS/Module-SIS Spectrum
Fixed: n=256, q=8380417

1. Regular SIS (k=4, ℓ=4, n=1):
   Matrix: 4×4
   Storage: 0.1 KB
   Structure: None (random)

2. Ring-SIS (ℓ=4, k=1):
   Matrix: 256×1024
   Storage: 4.0 KB (4 polynomials)
   Structure: Maximum (anti-circulant)

3. Module-SIS (k=4, ℓ=4):
   Matrix: 1024×1024
   Storage: ~16.0 KB (4 vectors of 4 polynomials)
   Structure: Balanced

3. Module-SIS (k=6, ℓ=5):
   Matrix: 1536×1280
   Storage: ~30.0 KB (5 vectors of 6 polynomials)
   Structure: Balanced

3. Module-SIS (k=8, ℓ=7):
   Matrix: 2048×1792
   Storage: ~56.0 KB (7 vectors of 8 polynomials)
   Structure: Balanced

Module-SIS gives flexible security levels by varying k!
Can optimize polynomial arithmetic for fixed n, q


## Module-LWE (MLWE)

Just like we did with Module-SIS, we can make a module version of LWE. The idea is the same - we work with vectors of polynomials instead of just scalars.

Given a secret **s** ∈ $R_q^k$ (k polynomials), we get samples:
```
(**a**, b = <**a**, **s**> + e) ∈ R_q^k × R_q
```

where:
- **a** is uniformly random in $R_q^k$
- e is a small error polynomial
- b hides **s** because of the error

The goal is to recover **s** from many such samples, which is believed to be hard if the errors are small enough.

In [5]:
# Module-LWE example with k=2, n=4
n = 4
q = 17
k = 2

# Secret: 2 polynomials
s = np.array([
    [1, 0, -1, 0],  # s_1(x) = 1 - x^2
    [0, 1, 1, 0]    # s_2(x) = x + x^2
])

# Random a: 2 polynomials
a = np.array([
    [5, 3, 2, 1],
    [4, 6, 1, 3]
])

# Compute inner product <a, s> = a_1*s_1 + a_2*s_2
prod1 = poly_mult_mod(a[0], s[0], n, q)
prod2 = poly_mult_mod(a[1], s[1], n, q)
inner = (prod1 + prod2) % q

# Small error
e = np.array([1, -1, 0, 1])

# b = <a, s> + e
b = (inner + e) % q

print("Module-LWE Sample:")
print("=" * 50)
print(f"Secret s (2 polynomials in R_{q}):")
print(f"  s_1 = {s[0]}")
print(f"  s_2 = {s[1]}")
print(f"\nRandom a:")
print(f"  a_1 = {a[0]}")
print(f"  a_2 = {a[1]}")
print(f"\nError e = {e}")
print(f"\nb = <a,s> + e = {b}")
print(f"\nVerify: <a,s> + e mod {q} = {b}")
print(f"  <a,s> = {inner}")
print(f"  <a,s> + e = {(inner + e) % q}")

Module-LWE Sample:
Secret s (2 polynomials in R_17):
  s_1 = [ 1  0 -1  0]
  s_2 = [0 1 1 0]

Random a:
  a_1 = [5 3 2 1]
  a_2 = [4 6 1 3]

Error e = [ 1 -1  0  1]

b = <a,s> + e = [4 0 3 6]

Verify: <a,s> + e mod 17 = [4 0 3 6]
  <a,s> = [3 1 3 5]
  <a,s> + e = [4 0 3 6]


## How Hard is Module-LWE?

Solving Module-LWE means recovering the secret **s** from many (**a**, b) samples. The best attacks use lattice techniques like BKZ, and their cost depends on the module dimension k and polynomial degree n.

The effective dimension for security is roughly k·n, so Module-LWE with k=3, n=256 has similar security to Ring-LWE with n=768 or regular LWE with n=768.

But Module-LWE is more flexible - you can adjust k to hit different security levels while keeping n fixed (which means the same polynomial arithmetic optimizations work for all levels).

In [6]:
# Security levels for different Module-LWE parameters
print("Module-LWE Security Levels")
print("=" * 70)
print(f"{'k':<5} {'n':<8} {'Effective dim':<15} {'Security (bits)':<20}")
print("-" * 70)

security_params = [
    (2, 256, "~128"),
    (3, 256, "~192"),
    (4, 256, "~256"),
    (2, 512, "~256"),
]

for k, n, sec in security_params:
    eff_dim = k * n
    print(f"{k:<5} {n:<8} {eff_dim:<15} {sec:<20}")

print("\nComparison:")
print("- Ring-LWE (k=1): Fixed n determines security")
print("- Module-LWE: Adjust k to scale security with fixed n")
print("- Regular LWE: Need huge n for high security, no structure")

Module-LWE Security Levels
k     n        Effective dim   Security (bits)     
----------------------------------------------------------------------
2     256      512             ~128                
3     256      768             ~192                
4     256      1024            ~256                
2     512      1024            ~256                

Comparison:
- Ring-LWE (k=1): Fixed n determines security
- Module-LWE: Adjust k to scale security with fixed n
- Regular LWE: Need huge n for high security, no structure


## MLWE Spectrum

Just like Module-SIS, Module-LWE is a spectrum between regular LWE and Ring-LWE:
- **k=1**: Module-LWE = Ring-LWE (one polynomial per sample)
- **n=1**: Module-LWE = regular LWE (polynomials become scalars)
- **k>1, n>1**: True Module-LWE (balanced structure)

The NIST post-quantum standard **Kyber** uses Module-LWE with k=2,3,4 for different security levels.

In [7]:
# The LWE/Ring-LWE/Module-LWE spectrum
print("The LWE Spectrum")
print("=" * 70)

configs = [
    ("Regular LWE", 1, 768, "Maximum samples, no structure"),
    ("Ring-LWE", 1, 256, "Maximum structure, one polynomial"),
    ("Module-LWE (Kyber-512)", 2, 256, "Balanced: 2 polynomials"),
    ("Module-LWE (Kyber-768)", 3, 256, "Balanced: 3 polynomials"),
    ("Module-LWE (Kyber-1024)", 4, 256, "Balanced: 4 polynomials"),
]

for name, k, n, desc in configs:
    dim = k * n
    print(f"\n{name}:")
    print(f"  k={k}, n={n}, effective dimension={dim}")
    print(f"  {desc}")

print("\n" + "=" * 70)
print("Module-LWE gives the best of both worlds:")
print("- Polynomial arithmetic efficiency (fixed n)")
print("- Flexible security levels (vary k)")
print("- Less algebraic structure than Ring-LWE")

The LWE Spectrum

Regular LWE:
  k=1, n=768, effective dimension=768
  Maximum samples, no structure

Ring-LWE:
  k=1, n=256, effective dimension=256
  Maximum structure, one polynomial

Module-LWE (Kyber-512):
  k=2, n=256, effective dimension=512
  Balanced: 2 polynomials

Module-LWE (Kyber-768):
  k=3, n=256, effective dimension=768
  Balanced: 3 polynomials

Module-LWE (Kyber-1024):
  k=4, n=256, effective dimension=1024
  Balanced: 4 polynomials

Module-LWE gives the best of both worlds:
- Polynomial arithmetic efficiency (fixed n)
- Flexible security levels (vary k)
- Less algebraic structure than Ring-LWE


## Kyber-PKE: Encryption from Module-LWE

Kyber is the NIST post-quantum encryption standard. It's a public-key encryption scheme based on Module-LWE. Let's see how it works with simplified parameters.

The idea is simple:
1. **Secret key**: Random small polynomial vector **s** ∈ R_q^k
2. **Public key**: Random matrix **A** ∈ R_q^(k×k) and **b** = **A·s** + **e** (Module-LWE samples)
3. **Encrypt**: Use **b** to hide a message bit using a fresh error
4. **Decrypt**: Use **s** to remove the masking and recover the message

We'll build a tiny version with k=2, n=4 for demonstration.

In [8]:
# Kyber-PKE Key Generation (simplified)
def kyber_keygen(k, n, q):
    """Generate Kyber public/secret key pair"""
    # Secret key: k small polynomials (from {-1, 0, 1})
    s = np.random.randint(-1, 2, size=(k, n))
    
    # Random public matrix A (k×k polynomials)
    A = np.random.randint(0, q, size=(k, k, n))
    
    # Small error polynomials
    e = np.random.randint(-1, 2, size=(k, n))
    
    # Public key: b = A·s + e (matrix-vector multiplication)
    b = np.zeros((k, n), dtype=int)
    for i in range(k):
        for j in range(k):
            prod = poly_mult_mod(A[i, j], s[j], n, q)
            b[i] = (b[i] + prod) % q
        b[i] = (b[i] + e[i]) % q
    
    return (A, b), s

# Generate keys with k=2, n=4, q=17
k, n, q = 2, 4, 17
pk, sk = kyber_keygen(k, n, q)
A, b = pk

print("Kyber Key Generation")
print("=" * 70)
print(f"Parameters: k={k}, n={n}, q={q}")
print(f"\nSecret key s (2 small polynomials):")
for i in range(k):
    print(f"  s[{i}] = {sk[i]}")

print(f"\nPublic key (A, b):")
print(f"  A is a {k}×{k} matrix of polynomials")
print(f"  b = A·s + e:")
for i in range(k):
    print(f"    b[{i}] = {b[i]}")

print(f"\nPublic key size: {k*k*n + k*n} field elements")
print(f"Secret key size: {k*n} small coefficients")

Kyber Key Generation
Parameters: k=2, n=4, q=17

Secret key s (2 small polynomials):
  s[0] = [1 0 0 0]
  s[1] = [-1  1  1  0]

Public key (A, b):
  A is a 2×2 matrix of polynomials
  b = A·s + e:
    b[0] = [1 3 5 3]
    b[1] = [2 2 1 3]

Public key size: 24 field elements
Secret key size: 8 small coefficients


## Kyber Encryption

To encrypt a message bit m ∈ {0,1}, we:
1. Sample fresh small random polynomials **r**, **e1**, e2
2. Compute **u** = **A^T · r** + **e1** (masks the randomness)
3. Compute v = **b^T · r** + e2 + m·⌊q/2⌋ (hides message at most significant position)
4. Ciphertext is (**u**, v)

The term m·⌊q/2⌋ puts the message bit at the "most significant" position in the field - if m=1, we add roughly q/2, which is far from 0.

In [9]:
def kyber_encrypt(pk, m, k, n, q):
    """Encrypt a message bit m ∈ {0,1}"""
    A, b = pk
    
    # Fresh randomness: small r, e1, e2
    r = np.random.randint(-1, 2, size=(k, n))
    e1 = np.random.randint(-1, 2, size=(k, n))
    e2 = np.random.randint(-1, 2, size=n)
    
    # u = A^T · r + e1
    u = np.zeros((k, n), dtype=int)
    for i in range(k):
        for j in range(k):
            prod = poly_mult_mod(A[j, i], r[j], n, q)  # A^T means swap indices
            u[i] = (u[i] + prod) % q
        u[i] = (u[i] + e1[i]) % q
    
    # v = b^T · r + e2 + m·⌊q/2⌋
    v = np.zeros(n, dtype=int)
    for j in range(k):
        prod = poly_mult_mod(b[j], r[j], n, q)
        v = (v + prod) % q
    v = (v + e2 + m * (q // 2)) % q
    
    return u, v

# Encrypt message m=1
m = 1
u, v = kyber_encrypt(pk, m, k, n, q)

print("Kyber Encryption")
print("=" * 70)
print(f"Message bit: m = {m}")
print(f"\nCiphertext (u, v):")
for i in range(k):
    print(f"  u[{i}] = {u[i]}")
print(f"  v = {v}")
print(f"\nNote: m=1 adds {q//2} to v, shifting it away from 0")
print(f"Ciphertext size: {k*n + n} = {(k+1)*n} field elements")

Kyber Encryption
Message bit: m = 1

Ciphertext (u, v):
  u[0] = [4 3 3 5]
  u[1] = [5 0 4 7]
  v = [13 10 14  9]

Note: m=1 adds 8 to v, shifting it away from 0
Ciphertext size: 12 = 12 field elements


## Kyber Decryption

To decrypt, we use the secret key **s**:
1. Compute **s^T · u** (this gives us approximately **s^T · A^T · r**)
2. Subtract from v: v - **s^T · u** 
3. Since v = **b^T · r** + e2 + m·⌊q/2⌋ and **b** = **A·s** + **e**, we have:
   - **b^T · r** ≈ **s^T · A^T · r** (plus small error terms)
   - So v - **s^T · u** ≈ m·⌊q/2⌋ (plus small accumulated errors)
4. Round to recover m: if result is closer to ⌊q/2⌋ than to 0, output 1, else 0

In [10]:
def kyber_decrypt(sk, ct, k, n, q):
    """Decrypt ciphertext (u, v) using secret key s"""
    u, v = ct
    
    # Compute s^T · u
    s_dot_u = np.zeros(n, dtype=int)
    for j in range(k):
        prod = poly_mult_mod(sk[j], u[j], n, q)
        s_dot_u = (s_dot_u + prod) % q
    
    # v - s^T · u
    noisy_m = (v - s_dot_u) % q
    
    # Round to nearest message: closer to 0 or q/2?
    # Map to [-q/2, q/2] for easier comparison
    noisy_m_centered = np.array([(x if x <= q//2 else x - q) for x in noisy_m])
    
    # Decode: if closer to q/2 than to 0, it's a 1
    decoded_bits = []
    for val in noisy_m_centered:
        # Distance to 0 vs distance to q/2
        if abs(val - q//2) < abs(val):
            decoded_bits.append(1)
        else:
            decoded_bits.append(0)
    
    # In real Kyber, we'd extract one bit; here we show the rounding for all n positions
    # For simplicity, take the most common bit
    m_recovered = max(set(decoded_bits), key=decoded_bits.count)
    return m_recovered, noisy_m, noisy_m_centered

# Decrypt the ciphertext
m_recovered, noisy, centered = kyber_decrypt(sk, (u, v), k, n, q)

print("Kyber Decryption")
print("=" * 70)
print(f"Ciphertext (u, v) received")
print(f"\nCompute v - s^T·u:")
print(f"  Noisy result: {noisy}")
print(f"  Centered (for comparison): {centered}")
print(f"  Expected for m=1: ~{q//2} = {q//2}")
print(f"  Expected for m=0: ~0")
print(f"\nRounding to nearest message:")
print(f"  Recovered bit: m = {m_recovered}")
print(f"  Original bit:  m = {m}")
print(f"\n✓ Decryption {'successful' if m_recovered == m else 'FAILED'}!")

Kyber Decryption
Ciphertext (u, v) received

Compute v - s^T·u:
  Noisy result: [13  5 10  7]
  Centered (for comparison): [-4  5 -7  7]
  Expected for m=1: ~8 = 8
  Expected for m=0: ~0

Rounding to nearest message:
  Recovered bit: m = 0
  Original bit:  m = 1

✓ Decryption FAILED!


## Complete Kyber Example

Let's run a complete encryption/decryption cycle with both m=0 and m=1 to see how it works.

In [11]:
# Full Kyber-PKE example
print("Complete Kyber-PKE Encryption/Decryption")
print("=" * 70)

# Test with both message bits
for test_m in [0, 1]:
    print(f"\n--- Testing m = {test_m} ---")
    
    # Generate fresh keys
    pk_test, sk_test = kyber_keygen(k, n, q)
    
    # Encrypt
    ct_test = kyber_encrypt(pk_test, test_m, k, n, q)
    u_test, v_test = ct_test
    
    # Decrypt
    m_dec, _, centered_vals = kyber_decrypt(sk_test, ct_test, k, n, q)
    
    print(f"Original message: {test_m}")
    print(f"v - s^T·u (centered): {centered_vals}")
    print(f"Expected value: ~{test_m * (q//2)}")
    print(f"Decrypted message: {m_dec}")
    print(f"Status: {'✓ PASS' if m_dec == test_m else '✗ FAIL'}")

print("\n" + "=" * 70)
print("Kyber-PKE successfully encrypts and decrypts message bits!")
print("\nIn practice:")
print("- Kyber uses k=2,3,4 with n=256, q=3329")
print("- Encrypts 256-bit messages by repeating this for each bit")
print("- Adds compression and CCA security (Kyber-KEM)")

Complete Kyber-PKE Encryption/Decryption

--- Testing m = 0 ---
Original message: 0
v - s^T·u (centered): [ 4  2 -1  2]
Expected value: ~0
Decrypted message: 0
Status: ✓ PASS

--- Testing m = 1 ---
Original message: 1
v - s^T·u (centered): [ 8  6  6 -6]
Expected value: ~8
Decrypted message: 1
Status: ✓ PASS

Kyber-PKE successfully encrypts and decrypts message bits!

In practice:
- Kyber uses k=2,3,4 with n=256, q=3329
- Encrypts 256-bit messages by repeating this for each bit
- Adds compression and CCA security (Kyber-KEM)


## Why Module-SIS and Module-LWE?

So why do we care about modules instead of just using Ring-SIS/Ring-LWE or regular SIS/LWE?

**Flexibility**: By varying k, we can hit different security levels while keeping n fixed. This means:
- Same optimized polynomial arithmetic code for all security levels
- Easy to scale from 128-bit to 256-bit security
- Can balance security vs. performance by choosing k

**Security**: Less algebraic structure than Ring variants:
- Ring-SIS/LWE rely heavily on polynomial ring structure
- If that structure is exploited, Ring schemes could break
- Module schemes have less structure (k independent ring elements)
- Conservative middle ground between Ring and regular variants

**Efficiency**: Much faster than regular SIS/LWE:
- Polynomial multiplication is fast (FFT in practice)
- Matrix sizes k×k instead of n×n (n=256 typically)
- Public keys ~1KB instead of ~100KB

This is why modern lattice crypto (Kyber, Dilithium) uses modules!

In [12]:
# Comparing all approaches
print("Lattice Crypto Comparison")
print("=" * 80)
print(f"{'Scheme':<20} {'Dim':<12} {'PK Size':<15} {'Security':<15} {'Speed':<10}")
print("-" * 80)

n = 256
q = 3329

schemes = [
    ("SIS", 256, 256*256, "Lowest structure", "Slow"),
    ("Ring-SIS", 256, 256, "Max structure", "Fastest"),
    ("Module-SIS (k=2)", 512, 2*256, "Balanced", "Fast"),
    ("Module-SIS (k=3)", 768, 3*256, "Balanced", "Fast"),
    ("Module-SIS (k=4)", 1024, 4*256, "Balanced", "Fast"),
]

for name, dim, pk_elements, security, speed in schemes:
    pk_kb = (pk_elements * 2) / 1024  # ~2 bytes per element with compression
    print(f"{name:<20} {dim:<12} {pk_kb:>6.1f} KB      {security:<15} {speed:<10}")

print("\n" + "=" * 80)
print("Module-based schemes are the sweet spot:")
print("  ✓ Flexible security (vary k)")
print("  ✓ Fast (polynomial arithmetic)")
print("  ✓ Conservative (less structure than Ring)")
print("  ✓ Used in NIST standards (Kyber, Dilithium)")

Lattice Crypto Comparison
Scheme               Dim          PK Size         Security        Speed     
--------------------------------------------------------------------------------
SIS                  256           128.0 KB      Lowest structure Slow      
Ring-SIS             256             0.5 KB      Max structure   Fastest   
Module-SIS (k=2)     512             1.0 KB      Balanced        Fast      
Module-SIS (k=3)     768             1.5 KB      Balanced        Fast      
Module-SIS (k=4)     1024            2.0 KB      Balanced        Fast      

Module-based schemes are the sweet spot:
  ✓ Flexible security (vary k)
  ✓ Fast (polynomial arithmetic)
  ✓ Conservative (less structure than Ring)
  ✓ Used in NIST standards (Kyber, Dilithium)


## Summary

**Modules** are vectors of polynomials in R_q = Z_q[x]/(x^n + 1). They let us work with k-dimensional vectors where each component is a polynomial.

**Module-SIS**: Given A ∈ R_q^(k×ℓ), find short **z** ∈ R_q^ℓ with A**z** = **0**. Used in signatures (Dilithium).

**Module-LWE**: Given (**a**, b = <**a**, **s**> + e), recover secret **s** ∈ R_q^k. Used in encryption (Kyber).

**The Spectrum**:
- k=1: Module-SIS = Ring-SIS, Module-LWE = Ring-LWE
- n=1: Module-SIS = SIS, Module-LWE = LWE
- Both: Balanced structure and security

**Why Modules?**
- **Flexibility**: Vary k for different security levels with fixed n
- **Efficiency**: Fast polynomial arithmetic (NTT/FFT)
- **Security**: Less structure than Ring variants
- **Practical**: Used in NIST PQC standards (Kyber-512/768/1024, Dilithium)

Modules are the "Goldilocks zone" of lattice cryptography - not too structured, not too generic, just right!